# FAKE REVIEW NODE EMBEDDING

In [1236]:
# importing all necessary libraries
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import os
import networkx as nx
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
#from node2vec import Node2Vec as n2v
sns.set()

%matplotlib inline

In [1237]:
import os, sys
from copy import deepcopy
import numpy as np
import pandas as pd

import torch as th
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

In [1238]:
print(f"Torch version: {th.__version__}.  CUDA version: {th.version.cuda}")

Torch version: 1.12.1+cu113.  CUDA version: 11.3


In [1239]:
dgl_version = f"dgl-cu{th.version.cuda.replace('.', '')}==0.7.2"
dgl_version

'dgl-cu113==0.7.2'

In [1240]:
# Follow instructions at https://www.dgl.ai/pages/start.html
!{sys.executable} -m pip install {dgl_version} -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html


In [1241]:
os.environ['DGLBACKEND'] = 'pytorch'
import dgl
from dgl import function as fn

In [1242]:
dgl.__version__

'0.7.2'

In [1243]:
from dgl.dataloading.pytorch import NodeDataLoader
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [1244]:
import torch
import torch.nn as nn
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
import argparse

from dgl.sampling import node2vec_random_walk

In [1245]:
#importing libraries
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import statistics
import math
import os
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
#nltk.download('all')
import re
pd.set_option('max_colwidth', 800)


#importing Libraies
import networkx as nx
import matplotlib.pyplot as plt
from numpy import genfromtxt
import collections
from collections import OrderedDict

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression,LogisticRegression, RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [1246]:
#loading dataset
df=pd.read_csv('/content/deceptive.csv')
df.head()

,deceptive,hotel,polarity,source,text,Class
0,truthful,conrad,positive,TripAdvisor,"We stayed for a one night getaway with family on a thursday. Triple AAA rate of 173 was a steal. 7th floor room complete with 44in plasma TV bose stereo, voss and evian water, and gorgeous bathroom(no tub but was fine for us) Concierge was very helpful. You cannot beat this location... Only flaw was breakfast was pricey and service was very very slow(2hours for four kids and four adults on a friday morning) even though there were only two other tables in the restaurant. Food was very good so it was worth the wait. I would return in a heartbeat. A gem in chicago... \n",1
1,truthful,hyatt,positive,TripAdvisor,"Triple A rate with upgrade to view room was less than $200 which also included breakfast vouchers. Had a great view of river, lake, Wrigley Bldg. & Tribune Bldg. Most major restaurants, Shopping, Sightseeing attractions within walking distance. Large room with a very comfortable bed. \n",1
2,truthful,hyatt,positive,TripAdvisor,"This comes a little late as I'm finally catching up on my reviews from the past several months:) A dear friend and I stayed at the Hyatt Regency in late October 2007 for one night while visiting a friend and her husband from out of town. This hotel is perfect, IMO. Easy check in and check out. Lovely, clean, comfortable rooms with great views of the city. I know this area pretty well and it's very convenient to many downtown Chicago attractions. We had dinner and went clubing with our friends around Division St.. We had no problems getting cabs back and forth to the Hyatt and there's even public transportation right near by but we didn't bother since we only needed cabs from and to the hotel. Parking, as is usual for Chicago, was expensive but we were able to get our car out quickly (h...",1
3,truthful,omni,positive,TripAdvisor,"The Omni Chicago really delivers on all fronts, from the spaciousness of the rooms to the helpful staff to the prized location on Michigan Avenue. While this address in Chicago requires a high level of quality, the Omni delivers. Check in for myself and a whole group of people with me was under 3 minutes, the staff had plentiful recommendations for dining and events, and the rooms are some of the largest you'll find at this price range in Chicago. Even the 'standard' room has a separate living area and work desk. The fitness center has free weights, weight machines, and two rows of cardio equipment. I shared the room with 7 others and did not feel cramped in any way! All in all, a great property! \n",1
4,truthful,hyatt,positive,TripAdvisor,"I asked for a high floor away from the elevator and that is what I got. The room was pleasantly decorated, functional and very clean. I didn't need a whole lot of service but when I did they were pleasant and prompt. I used the fitness center which was well equipped and everything was in working order. It is in a great location at one end of the Michigan Avenue shopping district. \n",1


In [1247]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [1248]:
df.shape

(1600, 6)

In [1249]:
df['Class'].value_counts()

0    800
1    800
Name: Class, dtype: int64

In [1250]:
# dropping unnecessary columns. 
df.drop(['deceptive', 'hotel', 'polarity', 'source'], axis = 1, inplace=True)
df.head()

,text,Class
0,"This is the hotel for the discriminating business traveler. It offers spacious rooms that are elegant and refined. With a cozy area complete with chairs and fireplace, they are the perfect place to entertain clients or discuss strategy for a business meeting with a colleague. The restaurant offers just the location to impress a perspective client. And all of this elegance and refinement in the heart of downtown Chicago\n",0
1,"What a wonderful experience, super great, helpful, friendly staff. Angie at the front desk made sure my husbands birthday would be great. Our room was beautiful. Angie suggested we go to Sullivans for dinner..perfect choice, make the reservation for us.(best seat in the house and a special dessert) wow...Above and beyond what any other hotel or staff has done for us before..Looking forward to staying there again & again. THANKS AGAIN \n",1
2,"I stayed at the Talbott twice in the last few weeks. I was very pleased with every facet of my stays. The staff, from doormen to front desk, to concierge, to housekeeping, were wonderful. The rooms were nicely sized and clean, and well stocked. I will return to the Talbott anytime! \n",1
3,"My boyfriend and I were amazed by the breathtaking view of Lake Michigan! We are from Texas so the view of the city was very important to us. We had no problems at this hotel... rooms, service, location were top notch. \n",1
4,"Rude, rude, rude! We stayed here on July 4th weekend. We drove from Michigan, we called customer service to get directions from I-95 w/c is a major highway to Chicago-- calling them from a cellphone, telling them we're in a middle of a storm and that we're lost....we were put on hold, forwarded to another person who did not know how to get to the hotel and we were on hold again. Giving up we decided to just exit the highway and try our luck at a 7-11 store. When we finally arrived at the hotel at 2 am no one was there to take care of our car so we looked for parking ourselves. Checking in we were not greeted by anyone, even the receptionist did not greet us!! no welcome to the knickerbocker, nothing! The man at the reception was rude, which makes me think what kind of customer service ...",1


In [1251]:
# Cleaning Raw tweets
def clean_text(text):
    
    #remove emails
    text = ' '.join([i for i in text.split() if '@' not in i])
    
    #remove web address
    text = re.sub('http[s]?://\S+', '', text)

    #remove brackets and parantheses (and removing the text inside the brackets and parantheses)
    text = re.sub("[\(\[].*?[\)\]]", "", text)

    #remove punctuations
    text = re.sub(r'[^\w\s]', ' ', text)
    
    #Filter to allow only alphabets and numbers
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    #remove double spaces 
    text = re.sub('\s+', ' ', text)
    
    return text

In [1252]:
# applying that into function.
df["clean_text"] = df['text'].apply(lambda x: clean_text(x))
df.head(10)

,text,Class,clean_text
0,"This is the hotel for the discriminating business traveler. It offers spacious rooms that are elegant and refined. With a cozy area complete with chairs and fireplace, they are the perfect place to entertain clients or discuss strategy for a business meeting with a colleague. The restaurant offers just the location to impress a perspective client. And all of this elegance and refinement in the heart of downtown Chicago\n",0,This is the hotel for the discriminating business traveler It offers spacious rooms that are elegant and refined With a cozy area complete with chairs and fireplace they are the perfect place to entertain clients or discuss strategy for a business meeting with a colleague The restaurant offers just the location to impress a perspective client And all of this elegance and refinement in the heart of downtown Chicago
1,"What a wonderful experience, super great, helpful, friendly staff. Angie at the front desk made sure my husbands birthday would be great. Our room was beautiful. Angie suggested we go to Sullivans for dinner..perfect choice, make the reservation for us.(best seat in the house and a special dessert) wow...Above and beyond what any other hotel or staff has done for us before..Looking forward to staying there again & again. THANKS AGAIN \n",1,What a wonderful experience super great helpful friendly staff Angie at the front desk made sure my husbands birthday would be great Our room was beautiful Angie suggested we go to Sullivans for dinner perfect choice make the reservation for us wow Above and beyond what any other hotel or staff has done for us before Looking forward to staying there again again THANKS AGAIN
2,"I stayed at the Talbott twice in the last few weeks. I was very pleased with every facet of my stays. The staff, from doormen to front desk, to concierge, to housekeeping, were wonderful. The rooms were nicely sized and clean, and well stocked. I will return to the Talbott anytime! \n",1,I stayed at the Talbott twice in the last few weeks I was very pleased with every facet of my stays The staff from doormen to front desk to concierge to housekeeping were wonderful The rooms were nicely sized and clean and well stocked I will return to the Talbott anytime
3,"My boyfriend and I were amazed by the breathtaking view of Lake Michigan! We are from Texas so the view of the city was very important to us. We had no problems at this hotel... rooms, service, location were top notch. \n",1,My boyfriend and I were amazed by the breathtaking view of Lake Michigan We are from Texas so the view of the city was very important to us We had no problems at this hotel rooms service location were top notch
4,"Rude, rude, rude! We stayed here on July 4th weekend. We drove from Michigan, we called customer service to get directions from I-95 w/c is a major highway to Chicago-- calling them from a cellphone, telling them we're in a middle of a storm and that we're lost....we were put on hold, forwarded to another person who did not know how to get to the hotel and we were on hold again. Giving up we decided to just exit the highway and try our luck at a 7-11 store. When we finally arrived at the hotel at 2 am no one was there to take care of our car so we looked for parking ourselves. Checking in we were not greeted by anyone, even the receptionist did not greet us!! no welcome to the knickerbocker, nothing! The man at the reception was rude, which makes me think what kind of customer service ...",1,Rude rude rude We stayed here on July th weekend We drove from Michigan we called customer service to get directions from I w c is a major highway to Chicago calling them from a cellphone telling them we re in a middle of a storm and that we re lost we were put on hold forwarded to another person who did not know how to get to the hotel and we were on hold again Giving up we decided to just exit the highway and try our luck at a store When we finally arrived at the hotel at am no one was t

In [1253]:
# dropping the uncleaned text column "text"
df = df.drop(['text'], axis=1)

In [1254]:
# renaming the column
df.rename(columns = {'clean_text':'text'}, inplace = True)
df.head()

,Class,text
0,0,This is the hotel for the discriminating business traveler It offers spacious rooms that are elegant and refined With a cozy area complete with chairs and fireplace they are the perfect place to entertain clients or discuss strategy for a business meeting with a colleague The restaurant offers just the location to impress a perspective client And all of this elegance and refinement in the heart of downtown Chicago
1,1,What a wonderful experience super great helpful friendly staff Angie at the front desk made sure my husbands birthday would be great Our room was beautiful Angie suggested we go to Sullivans for dinner perfect choice make the reservation for us wow Above and beyond what any other hotel or staff has done for us before Looking forward to staying there again again THANKS AGAIN
2,1,I stayed at the Talbott twice in the last few weeks I was very pleased with every facet of my stays The staff from doormen to front desk to concierge to housekeeping were wonderful The rooms were nicely sized and clean and well stocked I will return to the Talbott anytime
3,1,My boyfriend and I were amazed by the breathtaking view of Lake Michigan We are from Texas so the view of the city was very important to us We had no problems at this hotel rooms service location were top notch
4,1,Rude rude rude We stayed here on July th weekend We drove from Michigan we called customer service to get directions from I w c is a major highway to Chicago calling them from a cellphone telling them we re in a middle of a storm and that we re lost we were put on hold forwarded to another person who did not know how to get to the hotel and we were on hold again Giving up we decided to just exit the highway and try our luck at a store When we finally arrived at the hotel at am no one was there to take care of our car so we looked for parking ourselves Checking in we were not greeted by anyone even the receptionist did not greet us no welcome to the knickerbocker nothing The man at the reception was rude which makes me think what kind of customer service training this hotel has The guy ...


In [1255]:
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
print(sw_spacy)

{'never', 'show', 'three', 'then', 'via', 'ten', 'few', '’re', 'us', 'was', '‘m', 'same', 'make', 'may', 'more', 'whereafter', 'but', 'between', 'hence', 'beforehand', 'get', 'last', 'ourselves', 'four', 'latter', 'your', 'upon', 'almost', 'six', '’ve', 'onto', 'hereby', 'anything', '‘ll', 'else', 'however', 'others', 're', 'doing', 'other', 'much', 'wherein', 'which', '’m', 'toward', 'very', 'afterwards', 'see', 'least', 'again', 'they', 'fifteen', 'herein', 'less', 'side', 'regarding', 'two', 'nothing', 'often', 'off', 'amount', 'himself', 'done', 'me', 'nine', 'along', 'somewhere', 'although', 'am', 'a', 'too', 'seem', 'might', 'sixty', 'that', "n't", 'among', 'over', 'moreover', 'had', 'well', 'thereupon', 'bottom', 'who', 'anywhere', 'there', 'five', 'therein', 'no', 'this', 'third', 'every', 'an', 'wherever', 'whom', 'become', 'everything', 'below', 'former', 'unless', 'whether', 'perhaps', 'nobody', 'give', 'most', 'therefore', 'though', 'each', 'whatever', "'s", 'as', 'his', 'g

In [1256]:
import nltk
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
print(sw_nltk)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [1257]:
sw_nltk.extend(['th','stay'])
print(len(sw_nltk))

181


In [1258]:
# preprocessing the data (removing stopwords, convert the sentence to lower case, and also applying lemmatization to get meaningful words )
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
#stop_words = set(stopwords.words('english'))
ps=PorterStemmer()
wnl=WordNetLemmatizer()
def clean_lemma(review):
    review = review.lower()
    review = review.split()
    review = [wnl.lemmatize(word) for word in review if word not in sw_nltk]
    review = [wnl.lemmatize(word) for word in review if word not in sw_spacy]
    review = ' '.join(review)
    return review

In [1259]:
# using lemmatization
df['text']=df['text'].apply(clean_lemma)
df.head()

,Class,text
0,0,hotel discriminating business traveler offer spacious room elegant refined cozy area complete chair fireplace perfect place entertain client discus strategy business meeting colleague restaurant offer location impress perspective client elegance refinement heart downtown chicago
1,1,wonderful experience super great helpful friendly staff angie desk sure husband birthday great room beautiful angie suggested sullivan dinner perfect choice reservation u wow hotel staff u looking forward staying thanks
2,1,stayed talbott twice week pleased facet stay staff doorman desk concierge housekeeping wonderful room nicely sized clean stocked return talbott anytime
3,1,boyfriend amazed breathtaking view lake michigan texas view city important u problem hotel room service location notch
4,1,rude rude rude stayed july weekend drove michigan called customer service direction w c major highway chicago calling cellphone telling middle storm lost hold forwarded person know hotel hold giving decided exit highway try luck store finally arrived hotel care car looked parking checking greeted receptionist greet u welcome knickerbocker man reception rude think kind customer service training hotel guy find reservation frustrated checking able u direction maybe hour earlier room spectacular small probably little bigger walk closet risk


### FEATURE EXTRACTION

In [1260]:
x = df['text']
y = df['Class']

In [1261]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer()
corpii=cv.fit_transform(x) 

In [1262]:
feature_names = cv.get_feature_names()
#feature_names

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [1263]:
len(feature_names)

7905

In [1264]:
u1 = pd.DataFrame(corpii.toarray(), columns=feature_names)
u1.head()

,aaa,aaahed,aback,abassador,abd,ability,able,abound,abrupt,abruptly,...,yucky,yummo,yummy,yunan,yup,zagat,zest,zipped,zone,zoo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.080368,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### FEATURE SELECTION

In [1265]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# Two features with highest chi-squared statistics are selected
chi2_features = SelectKBest(chi2, k = 1000)
X_kbest_features = chi2_features.fit_transform(u1, y)

In [1266]:
chi_support = chi2_features.get_support()

In [1267]:
chi_feature = u1.loc[:,chi_support].columns.tolist()

In [1268]:
chi_feature

['accept',
 'accepted',
 'accomidations',
 'accommodated',
 'accommodation',
 'accomodations',
 'according',
 'accurate',
 'acknowledged',
 'acted',
 'adding',
 'address',
 'adult',
 'advantage',
 'advertises',
 'advisor',
 'affair',
 'affinia',
 'affordability',
 'agent',
 'al',
 'alive',
 'allegro',
 'allergenic',
 'allergy',
 'alley',
 'allowed',
 'amazing',
 'ambassador',
 'amenity',
 'angie',
 'angry',
 'anniversary',
 'annoying',
 'anthony',
 'anybody',
 'appointed',
 'april',
 'architectes',
 'arranging',
 'ask',
 'asleep',
 'associate',
 'assuming',
 'atmosphere',
 'attached',
 'attack',
 'attempted',
 'attended',
 'attending',
 'attraction',
 'attractive',
 'aug',
 'authorization',
 'av',
 'available',
 'ave',
 'aveda',
 'avenue',
 'award',
 'aware',
 'awful',
 'bacon',
 'badly',
 'baggage',
 'bakery',
 'ball',
 'bar',
 'bare',
 'bargain',
 'barking',
 'based',
 'basement',
 'basil',
 'bath',
 'bathroom',
 'battery',
 'beautiful',
 'beautifull',
 'beautifully',
 'beauty',
 'be

In [1269]:
X_kbest_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [1270]:
u3 = pd.DataFrame(X_kbest_features, columns = chi_feature)
u3.head()

,accept,accepted,accomidations,accommodated,accommodation,accomodations,according,accurate,acknowledged,acted,...,wouldnt,wrigley,wrong,wrote,year,yellow,yes,young,yummy,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1271]:
#from scipy.spatial.distance import cdist
#s = cdist(u3, u3, 'euclid')
#s

In [1272]:
#X1=pd.DataFrame(s)
#X1

In [1273]:
# taking average
y = np.average(u3)
y

0.002400953920696299

In [1274]:
adjmat_df=u3.copy()

In [1275]:
adjmat_df[adjmat_df < y]=0
adjmat_df[adjmat_df >= y]=1

In [1276]:
adjmat_df

,accept,accepted,accomidations,accommodated,accommodation,accomodations,according,accurate,acknowledged,acted,...,wouldnt,wrigley,wrong,wrote,year,yellow,yes,young,yummy,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1277]:
q = adjmat_df.to_numpy()
q

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [1278]:
# SOURCE : https://github.com/dmlc/dgl/issues/3364
src, dst = np.nonzero(q)

In [1279]:
src

array([   0,    0,    0, ..., 1599, 1599, 1599])

In [1280]:
dst

array([134, 140, 272, ..., 933, 965, 996])

In [1281]:
g = dgl.graph((src, dst))

In [1282]:
n_nodes = g.num_nodes()
n_nodes

1600

In [1283]:
nodes = g.nodes()
nodes

tensor([   0,    1,    2,  ..., 1597, 1598, 1599])

In [1284]:
n_nodes1 = g.number_of_nodes()
n_nodes1

1600

In [1285]:
g.ndata['feat'] = torch.tensor(X_kbest_features)
g.ndata['feat'] 

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [1286]:
n_train = int(n_nodes1 * 0.6)
n_val = int(n_nodes1 * 0.2)

In [1287]:
train_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(train_mask)
train_mask

tensor([False, False, False,  ..., False, False, False])

In [1288]:
val_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(val_mask)
val_mask

tensor([False, False, False,  ..., False, False, False])

In [1289]:
test_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(test_mask)
test_mask

tensor([False, False, False,  ..., False, False, False])

In [1290]:
train_mask[:n_train] = True

In [1291]:
val_mask[n_train:n_train + n_val] = True

In [1292]:
test_mask[n_train + n_val:] = True

In [1293]:
g.ndata['train_mask'] = train_mask

In [1294]:
g.ndata['val_mask'] = val_mask

In [1295]:
g.ndata['test_mask'] = test_mask

In [1296]:
s_array = df['Class'].to_numpy()
s_array

array([0, 1, 1, ..., 1, 0, 1])

In [1297]:
t1 = torch.tensor(s_array)
t1

tensor([0, 1, 1,  ..., 1, 0, 1])

In [1298]:
g.ndata["label"] =t1

In [1299]:
label = g.ndata["label"]

In [1300]:
label[train_mask]

tensor([0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
        1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
        0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
        1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
        0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1,
        0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,

In [1301]:
label[val_mask]

tensor([0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
        0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
        1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
        1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
        1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
        0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
        0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
        1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
        1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
        0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
        0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
        0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,

In [1302]:
label[test_mask]

tensor([0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
        0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
        1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
        1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
        1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0,

In [1303]:
train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [1304]:
val_mask

tensor([False, False, False,  ..., False, False, False])

In [1305]:
test_mask

tensor([False, False, False,  ...,  True,  True,  True])

In [1306]:
# Contruct a two-layer GNN model
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [1307]:
node_features = g.ndata['feat'].float()
node_labels = g.ndata['label']
train_mask = g.ndata['train_mask']
valid_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

In [1308]:
node_features

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [1309]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [1310]:
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=n_labels)
opt = torch.optim.Adam(model.parameters())

for epoch in range(50):
    model.train()
    # forward propagation by using all nodes
    logits = model(g, node_features)
    # compute loss
    loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
    # compute validation accuracy
    acc = evaluate(model, g, node_features, node_labels, valid_mask)
    print(acc)
    # backward propagation
    opt.zero_grad()
    loss.backward()
    opt.step()
    #print(loss.item())

    # Save model if necessary.  Omitted in this example.

0.5125
0.553125
0.60625


/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


0.65625
0.68125
0.715625
0.753125
0.765625
0.7875
0.809375
0.8375
0.853125
0.865625
0.865625
0.86875
0.875
0.884375
0.884375
0.884375
0.8875
0.8875
0.8875
0.890625
0.89375
0.896875
0.896875
0.896875
0.896875
0.9
0.9
0.90625
0.9125
0.909375
0.909375
0.909375
0.9125
0.9125
0.9125
0.9125
0.915625
0.915625
0.915625
0.915625
0.915625
0.91875
0.91875
0.91875
0.921875
0.921875
0.921875
